In [11]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import time
np.set_printoptions(suppress = True)
from Minimization import minimization
from concurrent.futures import ProcessPoolExecutor

In [2]:
T = 282

λ, η = 0.5, 0
b11, b22 = 1, 0.5

As11, As12, As13, As14 = 0.9, 0.0, 0.0, 0
As21, As22, As23, As24 = 0.0, 0.8, 0.0, 0
As31, As32, As33, As34 = 0.0, 0.0, 0.7, 0

Bs11, Bs21, Bs22, Bs31, Bs32, Bs33 = 4, 0, 3, 0, 0, 2

θ = np.array([λ, η, \
              b11, b22, \
              As11, As12, As13, As14, \
              As21, As22, As23, As24, \
              As31, As32, As33, As34, \
              Bs11, Bs21, Bs22, Bs31, Bs32, Bs33])

In [26]:
Ass = np.array([[As11, As12, As13],\
                    [As21, As22, As23],\
                    [As31, As32, As33]])
Aso = np.array([[As14],\
                [As24],\
                [As34]])
Bs =  np.array([[Bs11, 0,    0],\
                [Bs21, Bs22, 0],\
                [Bs31, Bs32, Bs33]])

μs = sp.linalg.solve(np.eye(3) - Ass, Aso) 
Σs = sp.linalg.solve_discrete_lyapunov(Ass, Bs@Bs.T)


In [3]:
def sim_VAR(A, B, X0, shock_num, T):

    X_series = np.zeros([X0.shape[0], T+1])
    X_series[:, [0]]= X0

    W_series = np.random.multivariate_normal(np.zeros(shock_num), np.diag(np.ones(shock_num)), [T+1]).T

    for t in range(T):
        X_series[:,[t+1]] = A @ X_series[:,[t]] + B @ W_series[:,[t+1]]

    return X_series, W_series

def sim_obs(θ):
    
    λ, η, b11, b22, As11, As12, As13, As14, As21, As22, As23, As24, As31, As32, As33, As34, Bs11, Bs21, Bs22, Bs31, Bs32, Bs33 = θ
    ones = np.ones([3,1])
    Az_true = np.array([[1, 1, 0],\
                        [0, λ, η],\
                        [0, 0, 1]])
    Bz_true = np.array([[b11, 0],\
                        [0, b22],\
                        [0,   0]])
    As_true = np.array([[As11, As12, As13, As14],\
                        [As21, As22, As23, As24],\
                        [As31, As32, As33, As34],\
                        [0.0,  0.0,  0.0,  1.0]])
    Bs_true = np.array([[Bs11, 0.0,  0.0],\
                        [Bs21, Bs22, 0.0],\
                        [Bs31, Bs32, Bs33],\
                        [0.0,  0.0,  0.0]])
    
    Ass = np.array([[As11, As12, As13],\
                    [As21, As22, As23],\
                    [As31, As32, As33]])
    Aso = np.array([[As14],\
                    [As24],\
                    [As34]])
    Bs =  np.array([[Bs11, 0,    0],\
                    [Bs21, Bs22, 0],\
                    [Bs31, Bs32, Bs33]])
    
    μs = sp.linalg.solve(np.eye(3) - Ass, Aso) 
    Σs = sp.linalg.solve_discrete_lyapunov(Ass, Bs@Bs.T)
        
    μz01 = 0
    Σz01 = 10
    Z01 = np.random.normal(μz01, Σz01)

    μz02 = η/(1-λ)
    Σz02 = b22**2/(1-λ**2)
    Z02 = np.random.normal(μz02, Σz02)

    Z0_true = np.array([[Z01],[Z02],[1]])
    S0_true = np.append(np.random.multivariate_normal(μs.flatten(), Σs),np.array(1)).reshape([-1,1])

    Z_series, Wz = sim_VAR(Az_true, Bz_true, Z0_true, 2, T)
    S_series, Ws = sim_VAR(As_true, Bs_true, S0_true, 3, T)
    obs_series = ones@Z_series[[0],:] + S_series[0:3,:]
    
    return obs_series, Z_series, S_series, Wz, Ws

In [35]:
obs_series, Z_series, S_series, Wz, Ws = sim_obs(θ)

In [18]:
θ_true = np.array([λ, η, \
              b11, b22, \
              As11, As12, As13, As14, \
              As21, As22, As23, As24, \
              As31, As32, As33, As34, \
              Bs11, Bs21, Bs22, Bs31, Bs32, Bs33])
n_simulation = 2
start_trials = 2

if __name__ == '__main__':
    batch_results = []
    for i in range(n_simulation):
        obs_series, Z_series, S_series, Wz, Ws = sim_obs(θ_true)
        obs_series = [obs_series for _ in range(start_trials)] 
        with ProcessPoolExecutor() as pool:
            results = pool.map(minimization, obs_series)
        results = [r for r in results]
        batch_results.append(results)

/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/14th Version, MLE simulation/Minimization.py:92: RuntimeWarning: invalid value encountered in log
  ll += (-0.5*obs_series.shape[0]*np.log(2*np.pi) - 0.5*np.log(np.linalg.det(Ω)) - 0.5*(obs[:,[t+1]] - D@μt).T@np.linalg.inv(Ω)@(obs[:,[t+1]] - D@μt))
/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/14th Version, MLE simulation/Minimization.py:92: RuntimeWarning: invalid value encountered in log
  ll += (-0.5*obs_series.shape[0]*np.log(2*np.pi) - 0.5*np.log(np.linalg.det(Ω)) - 0.5*(obs[:,[t+1]] - D@μt).T@np.linalg.inv(Ω)@(obs[:,[t+1]] - D@μt))
/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/14th Version, MLE simulation/Minimization.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  Z02 = η/(1-λ)
/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/14th Version, MLE simulation/Minimization.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  Σz02 = b22**2/(1-λ**2)
/User

In [22]:
batch_n = 1
with open('data_'+str(batch_n)+'.pkl', 'wb') as f:
       pickle.dump(batch_results, f)

In [23]:
with open('data_1.pkl', 'rb') as f:
    temp = pickle.load(f)

In [25]:
len(temp[0])

2

In [7]:
minimization(obs_series)

[      fun: 2163.623395956915
  hess_inv: <22x22 LbfgsInvHessProduct with dtype=float64>
       jac: array([-0.53009899, -0.70317583,  0.24228939,  0.3700734 , -0.64528649,
        -0.00222826, -0.07080416, -0.00145519, -3.0290721 ,  0.41063686,
         0.45642992, -0.29467628,  0.33160177, -0.35283847, -0.80904101,
         0.03096829,  0.0471573 ,  0.22532731,  0.25838745, -0.39808583,
        -0.35738594,  0.21655069])
   message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 1403
       nit: 55
      njev: 61
    status: 0
   success: True
         x: array([ 0.24545856,  0.12513604,  0.246189  ,  1.12942868,  0.9005119 ,
        -0.02509297, -0.08316784, -0.02934259,  0.0002632 ,  0.7836534 ,
         0.17478504, -0.01045534, -0.05907618,  0.06151134,  0.59565293,
         0.01615207,  4.04178071, -0.28551972,  2.92556576, -0.0229005 ,
         0.11282063,  1.89712169]),
 [array([ 0.5125182 , -0.07549691,  1.17702869,  0.36773443,  0.92463155,
         -0.06555561

In [33]:
def init_kf(θ):
    
    λ, η, b11, b22, As11, As12, As13, As14, As21, As22, As23, As24, As31, As32, As33, As34, Bs11, Bs21, Bs22, Bs31, Bs32, Bs33 = θ
    ones = np.ones([3,1])
    
    Ass = np.array([[As11, As12, As13],\
                    [As21, As22, As23],\
                    [As31, As32, As33]])
    Aso = np.array([[As14],\
                    [As24],\
                    [As34]])
    Bs =  np.array([[Bs11, 0,    0],\
                    [Bs21, Bs22, 0],\
                    [Bs31, Bs32, Bs33]])
    
    μs = sp.linalg.solve(np.eye(3) - Ass, Aso) 
    Σs = sp.linalg.solve_discrete_lyapunov(Ass, Bs@Bs.T)
    
    β = sp.linalg.solve(np.hstack([Σs@np.array([[1,1],[0,-1],[-1,0]]), ones]).T, np.array([[0,0,1]]).T)                                     
    γ1 = np.array([[1],[0],[0]]) - sp.linalg.inv(Σs)@ones/(ones.T@sp.linalg.inv(Σs)@ones)
    γ2 = np.array([[0],[1],[0]]) - sp.linalg.inv(Σs)@ones/(ones.T@sp.linalg.inv(Σs)@ones)
    γ3 = np.array([[0],[0],[1]]) - sp.linalg.inv(Σs)@ones/(ones.T@sp.linalg.inv(Σs)@ones)
    Γ = np.hstack([γ1, γ2, γ3])
    
    Z01 = β.T@(obs_series[:,[0]] - μs)
    Σz01 = 0
    Z02 = η/(1-λ)
    Σz02 = b22**2/(1-λ**2)
    S0 = Γ.T@(obs_series[:,[0]] - μs) + μs
    Σs0 = (1/(ones.T@np.linalg.inv(Σs)@ones))[0][0]
    
    μ0 = np.array([[Z01[0][0]],\
                   [Z02],\
                   [S0[0][0]],\
                   [S0[1][0]],\
                   [S0[2][0]],\
                   [1]])
    Σ0 = np.array([[Σz01,0,    0,   0,   0,   0],\
                   [0,   Σz02, 0,   0,   0,   0],\
                   [0,   0,    Σs0, Σs0, Σs0, 0],\
                   [0,   0,    Σs0, Σs0, Σs0, 0],\
                   [0,   0,    Σs0, Σs0, Σs0, 0],\
                   [0,   0,    0,   0,   0,   0]])    
    return μ0, Σ0, Ass, Σs

In [44]:
def Kalman_Filter(obs, D, F, A, B, μ0, Σ0):

    state_μ = np.zeros([A.shape[1], obs.shape[1]])
    state_μ[:,[0]] = μ0
    state_Σ = np.zeros([A.shape[1], A.shape[1], obs.shape[1]])
    state_Σ[:,:,0] = Σ0

    ll = 0

    for t in range(obs.shape[1]-1):
        μt = state_μ[:,[t]]
        Σt = state_Σ[:,:,t]
        KΣt = (A@Σt@D.T + B@F.T)@np.linalg.inv(D@Σt@D.T+F@F.T)
        state_μ[:,[t+1]] = A@μt + KΣt@(obs[:,[t+1]] - D@μt)
        state_Σ[:,:,t+1] = A@Σt@A.T + B@B.T - (A@Σt@D.T + B@F.T)@np.linalg.inv(D@Σt@D.T + F@F.T)@(D@Σt@A.T+F@B.T)

        Ω = D@Σt@D.T + F@F.T
        ll += (-0.5*obs_series.shape[0]*np.log(2*np.pi) - 0.5*np.log(np.linalg.det(Ω)) - 0.5*(obs[:,[t+1]] - D@μt).T@np.linalg.inv(Ω)@(obs[:,[t+1]] - D@μt))

    return state_μ, state_Σ ,ll

In [50]:
def ll(θ):

    λ, η, b11, b22, As11, As12, As13, As14, As21, As22, As23, As24, As31, As32, As33, As34, Bs11, Bs21, Bs22, Bs31, Bs32, Bs33 = θ

    A = np.array([[1,   1,   0,     0,     0,     0],\
                  [0,   λ,   0,     0,     0,     η],\
                  [0,   0,   As11,  As12,  As13,  As14],\
                  [0,   0,   As21,  As22,  As23,  As24],\
                  [0,   0,   As31,  As32,  As33,  As34],\
                  [0,   0,   0,     0,     0,     1]])
    B = np.array([[b11, 0,   0,     0,     0],\
                  [0,   b22, 0,     0,     0],\
                  [0,   0,   Bs11,  0,     0],\
                  [0,   0,   Bs21,  Bs22,  0],\
                  [0,   0,   Bs31,  Bs32,  Bs33],\
                  [0,   0,   0,       0,   0]])
    D = np.array([[1,   1,   As11,  As12,  As13,  As14],\
                  [1,   1,   As21,  As22,  As23,  As24],\
                  [1,   1,   As31,  As32,  As33,  As34]])
    F = np.array([[b11, 0,   Bs11,  0,     0],\
                  [b11, 0,   Bs21,  Bs22,  0],\
                  [b11, 0,   Bs31,  Bs32,  Bs33]])

    μ0, Σ0, _, _ = init_kf(θ)
    
    _, _, ll = Kalman_Filter(obs_series, D, F, A, B, μ0, Σ0)

    return -ll[0][0]

def callback(x):
    fobj = ll(x)
    θseries.append(x)
    llseries.append(fobj)


In [55]:

np.random.seed((os.getpid() * int(time.time())) % 123456789)
stable = False
n_trials = 0
while stable == False:
    start = θ + np.random.uniform(-0.2,0.2,22)
    try:
        μ0, Σ0, Ass, Σs = init_kf(start)
        n_trials +=1
        if (np.all(np.linalg.eigvals(Σ0)>=0)) & (np.all(abs(np.linalg.eigvals(Ass))<1) & (np.all(np.linalg.eigvals(Σs)>=0))):
            stable = True
        else:
            stable = False
    except: 
        n_trials +=1
        stable = False  
bnds = ((0,1),(-2,2),\
        (-2,2),(-2,2),\
        (-2,2),(-2,2),(-2,2),(-2,2),\
        (-2,2),(-2,2),(-2,2),(-2,2),\
        (-2,2),(-2,2),(-2,2),(-2,2),\
        (-5,5),(-5,5),(-5,5),(-5,5),(-5,5),(-5,5))
θseries = []
llseries = []
θ_opt = sp.optimize.minimize(ll, start, method = 'L-BFGS-B', bounds = bnds, callback = callback, tol=1e-6)  
   